# The Windows PE file format
There are tons of Python libraries for dissecting programs, one of the best is [lief](https://github.com/lief-project/LIEF).
The latter is also used inside `secml-malware` to perturb samples, as shown in the other tutorials.
Opening an executable is straight-forward:

In [5]:
import lief

exe_path = 'PATH TO YOUR EXE'
exe_object: lief.PE = lief.parse(exe_path)

Now, the `exe_object` contains all the information of the loaded program.
We can look for all the components. For instance, here is how you can read the header metadata:

In [6]:
print('DOS Header')
print(exe_object.dos_header)

print('PE Header')
print(exe_object.header)

print('Optional Header')
print(exe_object.optional_header)

print('Sections')
for s in exe_object.sections:
	print(s.name, s.characteristics_lists)

DOS Header
Magic:                        5a4d
Used Bytes In The LastPage:   90
File Size In Pages:           3
Number Of Relocation:         0
Header Size In Paragraphs:    4
Minimum Extra Paragraphs:     0
Maximum Extra Paragraphs:     ffff
Initial Relative SS:          0
Initial SP:                   b8
Checksum:                     0
Initial IP:                   0
Initial Relative CS:          0
Address Of Relocation Table:  40
Overlay Number:               0
OEM id:                       0
OEM info:                     0
Address Of New Exe Header:    100

PE Header
Signature:                    50 45 0 0 
Machine:                      I386
Number Of Sections:           5
Pointer To Symbol Table:      0
Number Of Symbols:            0
Size Of Optional Header:      e0
Characteristics:              EXECUTABLE_IMAGE - CHARA_32BIT_MACHINE
Time Date Stamp:              56ac266b

Optional Header
Magic:                           10b
Major Linker Version:            a
Minor Linker Version:

This library is also very useful for manipulating the EXEs.
For instance, in few lines of code you can add sections to a program.

In [8]:
# Name your new section. Size constraint: up to 8 bytes at maximum!
new_section : lief.PE.Section = lief.PE.Section()
new_section.name = '.newsec'
new_section.content = [ord(i) for i in "This is my newly created section"]
new_section.characteristics = lief.PE.SECTION_CHARACTERISTICS.MEM_DISCARDABLE
exe_object.add_section(new_section)

# New section in place! Now we use lief to rebuild the binary.
builder = lief.PE.Builder(exe_object)
builder.build()
exe_object = lief.PE.parse(builder.get_build())
print('Sections')
for s in exe_object.sections:
	print(s.name, s.characteristics_lists)
builder.write('new_exe.file')

Sections
.text {SECTION_CHARACTERISTICS.CNT_CODE, SECTION_CHARACTERISTICS.MEM_EXECUTE, SECTION_CHARACTERISTICS.MEM_READ}
.rdata {SECTION_CHARACTERISTICS.CNT_INITIALIZED_DATA, SECTION_CHARACTERISTICS.MEM_READ}
.data {SECTION_CHARACTERISTICS.CNT_INITIALIZED_DATA, SECTION_CHARACTERISTICS.MEM_READ, SECTION_CHARACTERISTICS.MEM_WRITE}
.rsrc {SECTION_CHARACTERISTICS.CNT_INITIALIZED_DATA, SECTION_CHARACTERISTICS.MEM_READ}
.reloc {SECTION_CHARACTERISTICS.CNT_INITIALIZED_DATA, SECTION_CHARACTERISTICS.MEM_DISCARDABLE, SECTION_CHARACTERISTICS.MEM_READ}
.newsec {SECTION_CHARACTERISTICS.MEM_DISCARDABLE}
.newsec {SECTION_CHARACTERISTICS.MEM_DISCARDABLE}


As you can see, the new section appeared as last one.
More information on how to use lief on the [documentation of the library](https://lief-project.github.io/doc/stable/index.html).